# 05b - Mapping des noms d'équipes vers IDs numériques

Ce notebook transforme les noms d'équipes dans `matches.csv` en IDs numériques.

**Prérequis** : 
- `04-concat-and-index.ipynb` doit avoir été exécuté
- `05-mapping-teams-romain.ipynb` doit avoir créé `teams_mapping.json`

**Entrées** :
- `data/processed/matches.csv` (avec noms d'équipes)
- `data/processed/teams.csv`
- `data/reference/teams_mapping.json`

**Sortie** :
- `data/processed/matches.csv` (avec IDs numériques)

In [1]:
import pandas as pd
import json
from pathlib import Path

# Vérifier que teams_mapping.json existe
teams_mapping_path = Path('../data/reference/teams_mapping.json')
if not teams_mapping_path.exists():
    raise FileNotFoundError(
        "teams_mapping.json n'existe pas. "
        "Exécutez d'abord 05-mapping-teams-romain.ipynb pour le créer."
    )

# Charger les fichiers
with open(teams_mapping_path, 'r', encoding='utf-8') as f:
    teams_mapping = json.load(f)

df_matches = pd.read_csv('../data/processed/matches.csv')
df_teams = pd.read_csv('../data/processed/teams.csv')

print(f"Mapping JSON : {len(teams_mapping)} équipes")
print(f"Matches : {len(df_matches)} lignes")
print(f"Teams : {len(df_teams)} équipes")

Mapping JSON : 231 équipes
Matches : 7427 lignes
Teams : 264 équipes


In [2]:
# Construire le mapping complet: nom/alias → id_team
name_to_id = {}

for _, row in df_teams.iterrows():
    team_id = row['id_team']
    nom = row['nom_standard']

    # Ajouter le nom standard et ses variantes
    name_to_id[nom] = team_id
    name_to_id[nom.lower()] = team_id
    name_to_id[nom.upper()] = team_id
    name_to_id[nom.title()] = team_id

    # Ajouter les aliases depuis teams_mapping.json
    if nom in teams_mapping:
        for alias in teams_mapping[nom].get('aliases', []):
            name_to_id[alias] = team_id
            name_to_id[alias.lower()] = team_id
            name_to_id[alias.upper()] = team_id

print(f"Mapping créé : {len(name_to_id)} entrées")

Mapping créé : 817 entrées


In [3]:
def map_name_to_id(name):
    if pd.isna(name):
        return None
    name = str(name).strip()
    return (name_to_id.get(name) or
            name_to_id.get(name.lower()) or
            name_to_id.get(name.title()) or
            name_to_id.get(name.upper()))

# Garder les noms originaux
df_matches['home_team_name'] = df_matches['home_team_id'].copy()
df_matches['away_team_name'] = df_matches['away_team_id'].copy()

# Mapper vers IDs
df_matches['home_team_id'] = df_matches['home_team_name'].apply(map_name_to_id)
df_matches['away_team_id'] = df_matches['away_team_name'].apply(map_name_to_id)

# Vérifier les équipes non mappées
unmapped_home = df_matches[df_matches['home_team_id'].isna()]['home_team_name'].unique()
unmapped_away = df_matches[df_matches['away_team_id'].isna()]['away_team_name'].unique()
unmapped = set(unmapped_home) | set(unmapped_away)

if unmapped:
    print(f"⚠️ Équipes non mappées ({len(unmapped)}):")
    for name in sorted(unmapped):
        print(f"   - {name}")
    raise ValueError("Certaines équipes n'ont pas été mappées. Mettez à jour teams_mapping.json")
else:
    print("✅ Toutes les équipes ont été mappées vers des IDs")

# Convertir en entier et nettoyer
df_matches['home_team_id'] = df_matches['home_team_id'].astype('Int64')
df_matches['away_team_id'] = df_matches['away_team_id'].astype('Int64')
df_matches = df_matches.drop(columns=['home_team_name', 'away_team_name'])

✅ Toutes les équipes ont été mappées vers des IDs


In [4]:
# Export
df_matches.to_csv('../data/processed/matches.csv', index=False)
print(f"✅ Export : matches.csv avec IDs numériques")
print(f"   - {len(df_matches)} matchs")
print(f"   - home_team_id : min={df_matches['home_team_id'].min()}, max={df_matches['home_team_id'].max()}")
print(f"   - away_team_id : min={df_matches['away_team_id'].min()}, max={df_matches['away_team_id'].max()}")

print(f"\nAperçu:")
df_matches[['id_match', 'home_team_id', 'away_team_id', 'home_result', 'away_result', 'edition']].head(10)

✅ Export : matches.csv avec IDs numériques
   - 7427 matchs
   - home_team_id : min=1, max=264
   - away_team_id : min=1, max=264

Aperçu:


,id_match,home_team_id,away_team_id,home_result,away_result,edition
0,1,84,149,4,1,1930
1,2,246,23,3,0,1930
2,3,261,32,2,1,1930
3,4,188,179,3,1,1930
4,5,11,84,1,0,1930
5,6,46,149,3,0,1930
6,7,261,27,4,0,1930
7,8,246,177,3,0,1930
8,9,252,179,1,0,1930
9,10,46,84,1,0,1930
